# `DeepGalerkin`-model: solving PDEs with NNs

Welcome! In this tutorial you'll learn how to solve partial differential equations (PDEs) with neural networks using `DeepGalerkin`-model from `BatchFlow`. The notebook
* starts with general setup of the PDE solved by `DeepGalerkin`
* then explains in detail how the model is configured
* finally, demonstrates how to use the model for solving common pde-problems, including *2nd-order ordinary differential equation*, *heat-equation* and *wave-equation*.

To get full understanding of `DeepGalerkin`, we suggest you to read this tutorial starting from the problem setup. The alternative is to jump straight to the working examples.

**Note**: `DeepGalerkin` is written in [TensorFlow](https://www.tensorflow.org/). Throughout the notebook shorthand `tf` will stand for `TensoFlow`.

## Setup of the PDE-problem

Currently, `DeepGalerkin` supports equations of up to the second order with constant coefficients with `Dirichlet` boundary and initial conditions. In general form, the PDE-
problem looks as follows:
<a id='eq'></a>
\begin{align}
\sum_i a_i \frac{\partial u}{\partial x_i} + \sum_{i j} b_{i j} \frac{\partial^2 u}{\partial x_i \partial x_i}  = Q(x). \\
\end{align}
Where $a_{ij}, b_i $ -  arbitrary constants. PDE is solved on the domain
<a id='dom'></a>
\begin{align*}
\Omega = [b_0^1, b_1^1] \times \dots \times [b_0^n, b_1^n] ,\quad u: \mathcal{R}^n \rightarrow \mathcal{R}
\end{align*}
$x$ is $(x_1, \dots, x_n)$. In many tasks, $x_n$ represents time-dimension $t$. In that case, the initial conditions fix the initial state and evolution rate of the system:
<a id='inc_0'></a>
\begin{align*}
u(x_1, \dots, x_{n-1}, b_0^n) = u_0(x_1, \dots, x_{n-1}),
\end{align*}
<a id='inc_1'></a>
\begin{align*}
\frac{\partial u(x_1, \dots, x_{n-1}, b_0^n)}{\partial t} = u_0'(x_1, \dots, x_{n-1}).
\end{align*}
When only first-order time derivative is presented in lhs of PDE, only the initial value of system is needed:
\begin{align*}
u(x_1, \dots, x_{n-1}, b_0^n) = u_0(x_1, \dots, x_{n-1}),
\end{align*}
With `Dirichlet`-boundary conditions:
\begin{align*}
u(\sigma \Omega) = u_{\sigma}.
\end{align*}

The main idea of `DeepGalerkin` is to fit the parameters $\theta$ of network $net(x; \theta)$ so that the difference between left-hand-side (lhs) of the PDE and right-hand-size (rhs) be small:
\begin{align*}
Loss(\theta) = \int \limits_{\Omega} L\left(\sum_i a_i \frac{\partial net(x; \theta)}{\partial x_i} + \sum_{i j} b_{i j} \frac{\partial^2 net(x; \theta)}{\partial x_i \partial x_i} - Q(x)\right)  \mathcal{P}( d x) \rightarrow \min\limits_{\theta}.
\end{align*}

**Note:** In practice, $Loss(\theta)$ is estimated on a sample(batch) of points $\{(x^i_1,\dots, x^i_n)\}_{i=1,\dots,N}$.
As the distribution $\mathcal{P}$ is not fixed, any sampling scheme can be used.

## Configuring `DeepGalerkin`

`DeepGalerkin`-model is configured with a configuration-dict and comes down to
* describing lhs of the [equation](#eq): define coefficients $a_{i}, b_{ij}$. This corresponds to the key `form`:
```
'form' = {'d1': v, 'd2': m}
```
where `v` is a vector of $a_i$ and $m$ is a matrix of $b_{ij}$. The number of coefficients in $v$ (alternatively, rows/lines in $m$) defines the dimensionality of the problem.
* setting rhs ($Q$) of the [equation](#eq):
```
Q = tf_callable
```
where `tf_callable` is a callable, that accepts `tf`-tensor of shape `(batch_size, pde_dimensionality)` and returns a `tf`-tensor of shape `(batch_size, )`.
* defining the [domain](#dom) in form
```
domain = [[b_0^1, b_1^1],...,[b_0^n, b_1^n]]
```

## Solving common PDEs with `DeepGalerkin`